## IMPORT PANDAS AND OPEN ADN CROSS WALK TABLES

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import re
## set the display max to max
pd.set_option('display.max_rows', None)

# opening the crosswalk table
new_cross = pd.read_csv('new_crosswalk.csv')
ridership_crosswalk = pd.read_csv('ridership_stationID.csv')

## Reading the data

In [ ]:
turnstiles_filename = '2019/Turnstiles_each_2019.csv'
turnstiles = pd.read_csv(turnstiles_filename)
#####################
turnstiles.columns.values

## Reading the data

In [ ]:
time_counts = turnstiles['Time'].value_counts().count()
# alot of differnt times even in the middle of the 4 hour intervals so not all times are in whole hours order
# some are in between hours, minutes nad even seconds ex. 16:22:00 23:30:00, 11:14:43

In [ ]:
turnstiles.info()

## assigning data type 'datetime' with formats to column 'Date'  for later use
## fix the error and column naming 'Exits'

In [ ]:
# chang column 'Date' type to datetime format to compare it easier later
turnstiles['Date'] = pd.to_datetime(turnstiles['Date'], format = '%m/%d/%Y') 
#  found problem with the column 'Exists ' having too many trailling space so to rename it
turnstiles.rename(columns={'Exits                                                     ':'EXITS'}, inplace=True)
turnstiles.columns.values # check if fixed

In [ ]:
# get only dates within the desired year
turnstiles = turnstiles.loc[(pd.to_datetime(turnstiles['Date'], format='%m/%d/%Y') >= pd.to_datetime('01/31/2022', format='%m/%d/%Y'))& (pd.to_datetime(turnstiles['Date'], format='%m/%d/%Y') <= pd.to_datetime(date_up_limit, format='%m/%d/%Y'))]


## checking and save missing dates within the year

In [ ]:
# funciton to get all days in a year
def get_all_dates_in_year(year):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    delta = timedelta(days=1)
    all_dates = []

    current_date = start_date
    while current_date <= end_date:
        all_dates.append(current_date)
        current_date += delta

    return all_dates


# Get all dates in the year and store in a dataframe
dates_list = get_all_dates_in_year(year)
dates_df = pd.DataFrame(dates_list, columns=['Date'])


## Get unique values from 'Date' column
# change 'turnstiles' to the dataframe
dates_in_data = turnstiles['Date'].unique()

dates_in_data = pd.DataFrame({'Date': list(dates_in_data)})

dates_difference = dates_df.loc[~dates_df['Date'].isin(dates_in_data['Date']), 'Date']
dates_difference = pd.DataFrame(columns= ['Missing Dates'])
dates_difference['Missing Dates'] = dates_df.loc[~dates_df['Date'].isin(dates_in_data['Date']), 'Date']

#sort the dates and save it
#dates_difference.sort_values(['Missing Dates'], ascending= True , inplace=True)
#dates_difference.reset_index(drop = True, inplace = True)
#dates_difference.to_csv(missing_dates_fn, index = False)

## assigning the corresponding station complex ID to each row

In [ ]:
#Create special name to use crosswalk table
# creating a new column "unique_ID" based on the station, line name and division
turnstiles["unique_ID"] = turnstiles[["Station", "Line Name", "Division"]].apply("-".join, axis=1)

#merges the two tables
turnstiles = pd.merge(turnstiles, new_cross, on='unique_ID', how="left")


## dropping null values in the station_complex_id column

In [ ]:
## dropping all rows with 'null' value in 'MTA Station ID' column
nulIndex = turnstiles[turnstiles['station_complex_id'].isnull()].index
turnstiles.drop(nulIndex , inplace=True)


In [ ]:
# using the created crosswalk table to add geograpic info
turnstiles = pd.merge(turnstiles, new_cross, on='unique_ID', how="left")


##  'TURNSTILES' ridership counts every 4hrs by subtracting the sorted previous row

In [ ]:
## getting net entries and net exist method 

# 1. sort the rows by their unique turnstiles and data, abnd time in ascending order

turnstiles_4h = turnstiles[['station_complex_id', 'unique_ID', 'C/A','Unit','SCP','Date', 'Time', 'Entries', 'EXITS']].copy()
turnstiles_4h.sort_values(['C/A','Unit','SCP','Date', 'Time'], ascending= True , inplace=True)
turnstiles_4h.reset_index(drop = True, inplace = True)

# # 2. subtract every 4 hours (subtract from the previous row)
net_entries = turnstiles_4h.Entries - turnstiles_4h.Entries.shift(1)
turnstiles_4h['net_entries'] = net_entries.fillna(0).astype(int)

net_exits = turnstiles_4h.EXITS - turnstiles_4h.EXITS.shift(1)
turnstiles_4h['net_exits'] = net_exits.fillna(0).astype(int)

# Eliminating extreme values and sort 
turnstiles_4h = turnstiles_4h[(turnstiles_4h.net_entries < 20000) & (turnstiles_4h.net_entries >= 0) & (turnstiles_4h.net_exits < 10000) &  (turnstiles_4h.net_exits >= 0)]
turnstiles_4h = turnstiles_4h.loc[turnstiles_4h['Date'] != date_low_limit ]
turnstiles_4h.sort_values(['C/A','Unit','SCP','Date', 'Time'], ascending= True , inplace=True)

## saving the result to a csv file
turnstiles_4h.to_csv(turnstile_each_filename, index = False)


## total combined all stations 'TURNSTILES' riderships counts for each day

In [ ]:
##  total by daily
turnstiles_daily = turnstiles_4h.groupby(['Date'])[['net_entries', 'net_exits']].sum().reset_index()
turnstiles_daily.sort_values(['Date'], ascending= True , inplace=True)

daily_sorted_entries = turnstiles_daily.sort_values(['net_entries'], ascending= False)
turnstiles_daily.rename(columns={'net_entries':'ridership'}, inplace=True)
#turnstiles_daily.to_csv(turnstiles_daily_fn, index = False)